# Ansible

**Sources**

- [Building an inventory](https://docs.ansible.com/ansible/latest/getting_started/get_started_inventory.html).

## Hello world

To make ansible work we need:

- Control node with ssh client and ansible installed.
- Managed node with ssh server, and python interpreter.
- Provide access from control node to managed node.

The role of the nodes in our examples will be to play docker containers. The following cell creates docker containers and configures access between them.

In [19]:
docker network create ansible_test_network
docker run -itd --rm --name ansible_managed_node --network ansible_test_network python:3.10-alpine
docker run -itd --rm --name ansible_control_node --network ansible_test_network python:3.10-alpine

# openssh instalation
docker exec ansible_managed_node sh -c "
apk add -q openssh
ssh-keygen -A
mkdir /root/.ssh
/usr/sbin/sshd -D &
" > /dev/null

docker exec ansible_control_node sh -c "
apk add openssh-client
ssh-keygen -t rsa -N \"\" -f /root/.ssh/id_rsa
pip3 install ansible
# Add to the config statement that we don't need to enter if we want to save the new host.
echo -e \"Host *\n\tStrictHostKeyChecking no\n\n\" > ~/.ssh/config
" > /dev/null

export public_key=$(docker exec ansible_control_node cat /root/.ssh/id_rsa.pub)
docker exec ansible_managed_node sh -c "echo \"$public_key\" >> /root/.ssh/authorized_keys"

ade562fb2daef876f87e02abd24407aa9907dbc1dd46a11bbe9ba4d9ef369801
d83689eadce86220ce6b96bb3f39aa3768a484fdfc7323fb7f3480311d28aaf5
9c071a4146ea47505c369727dc96c7c291b4c3bc2d19d71da8a961bf4e16f44e
Created directory '/root/.ssh'.

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


**Note** don't forget to stop containres after all.

In [18]:
docker stop ansible_managed_node ansible_control_node
docker network rm ansible_test_network

ansible_managed_node
ansible_control_node
ansible_test_network


### Inventory

Inventories organise managed nodes into centralised files that provide Ansible with system information and network locations. An inventory file allows Ansible to manage a large number of hosts with a single command. So it's a file in which you list all the managed nodes.

Now in the control node we need to create an ansible config `inventory.ini` where we just specify managed node.

In [20]:
docker exec ansible_control_node sh -c  "
cat << EOF > inventory.ini
[myhosts]
ansible_managed_node
EOF"

Finally, try pinging the managed nodes using the command `ansible myhosts -m ping -i inventory.ini`.

In [21]:
docker exec ansible_control_node ansible myhosts -m ping -i inventory.ini

[WARNING]: Platform linux on host ansible_managed_node is using the discovered
Python interpreter at /usr/local/bin/python3.10, but future installation of
another Python interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.17/reference_appendices/interpreter_discovery.html for more information.
ansible_managed_node | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/local/bin/python3.10"
    },
    "changed": false,
    "ping": "pong"
}


As a result, we got the message `ansible_host | SUCCESS ...`, which tells us that we successfully ping the managed node.

### Playbook

Playbooks are files with instructions. So below is a playbook that creates a play called `My first play`. This play will ping all hosts and then print a message.

In [22]:
docker exec ansible_control_node sh -c  "
cat << EOF > playbook.yaml
- name: My first play
  hosts: myhosts
  tasks:
   - name: Ping my hosts
     ansible.builtin.ping:

   - name: Print message
     ansible.builtin.debug:
      msg: Hello world"

So now you can use the `ansible-playbook` command to play the playbook we have just created.

In [23]:
docker exec ansible_control_node ansible-playbook -i inventory.ini playbook.yaml


PLAY [My first play] ***********************************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Platform linux on host ansible_managed_node is using the discovered
Python interpreter at /usr/local/bin/python3.10, but future installation of
another Python interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.17/reference_appendices/interpreter_discovery.html for more information.
ok: [ansible_managed_node]

TASK [Ping my hosts] ***********************************************************
ok: [ansible_managed_node]

TASK [Print message] ***********************************************************
ok: [ansible_managed_node] => {
    "msg": "Hello world"
}

PLAY RECAP *********************************************************************
ansible_managed_node       : ok=3    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



So there are tasks: `Ping my hosts` and `Print message`, just as we specified.